# Calculating potential market revenue

In [136]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Import dataset

In [137]:
price_df = pd.read_csv('../data/clean_data/wholesale_price_2022.csv', index_col=0)
price_df = price_df[['date', 'datetime', 'hour', 'gb_price_eur', 'PUN', 'NORD', 'CNOR', 'CSUD', 'SUD', 'SARD', 'SICI']]
price_df.head()

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI
0,2022-01-01,2022-01-01 00:00:00,1,89.24,170.28,170.28,170.28,170.28,170.28,170.28,170.28
1,2022-01-01,2022-01-01 01:00:00,2,89.24,155.72,155.72,155.72,155.72,155.72,155.72,155.72
2,2022-01-01,2022-01-01 02:00:00,3,108.52,147.09,147.09,147.09,147.09,147.09,147.09,147.09
3,2022-01-01,2022-01-01 03:00:00,4,57.12,91.00,91.00,91.00,91.00,91.00,91.00,91.00
4,2022-01-01,2022-01-01 04:00:00,5,83.29,104.00,104.00,104.00,104.00,104.00,104.00,104.00


### Daily spread

In [ ]:
daily_max = price_df.groupby('date').max().reset_index()
daily_min = price_df.groupby('date').min().reset_index()
daily_max.head()

In [139]:
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('GB', 'Italy'),
    shared_xaxes=True
)

fig.add_trace(go.Scatter(
    y=daily_min.gb_price_eur,
    x=daily_min.datetime,
    line=dict(color='crimson', width=2),
    name='GB min'
), row=1, col=1)
fig.add_trace(go.Scatter(
    y=daily_max.gb_price_eur,
    x=daily_max.datetime,
    line=dict(color='green', width=2),
    name='GB max'
), row=1, col=1)
fig.add_trace(go.Scatter(
    y=daily_min.PUN,
    x=daily_min.datetime,
    line=dict(color='crimson', width=2),
    name='Italy PUN min'
), row=2, col=1)
fig.add_trace(go.Scatter(
    y=daily_max.PUN,
    x=daily_max.datetime,
    line=dict(color='green', width=2),
    name='Italy PUN max'
), row=2, col=1)

fig.update_layout(
    title='Daily highs and lows for British and Italian Day-Ahead markets',
    template='ggplot2',
    width=1200,
    height=500,
)
fig.update_yaxes(title='€/MWh')

fig.write_image('../figures/gb-italy-da.jpeg', scale=5, engine='orca')
fig

### Single cycle 1 hour

In [140]:
single_cycle_dict = {
    'NORD': (daily_max.NORD - daily_min.PUN).sum(),
    'CNOR': (daily_max.CNOR - daily_min.PUN).sum(),
    'CSUD': (daily_max.CSUD - daily_min.PUN).sum(),
    'SUD': (daily_max.SUD - daily_min.PUN).sum(),
    'SICI': (daily_max.SICI - daily_min.PUN).sum(),
    'SARD': (daily_max.SARD - daily_min.PUN).sum(),
}

print(f'GB revenue: {(daily_max.gb_price_eur - daily_min.gb_price_eur).sum()}')
single_cycle_df = pd.DataFrame(single_cycle_dict, index=['revenue']).transpose().round(2)
single_cycle_df

GB revenue: 50551.87


,revenue
NORD,59735.26
CNOR,59766.75
CSUD,58052.87
SUD,57291.98
SICI,58018.73
SARD,57732.57


In [141]:
fig = px.bar(
    single_cycle_df,
    color=single_cycle_df.revenue.values,
    color_continuous_scale='solar',
    color_continuous_midpoint=59e3,
    text_auto='.3s'
)

fig.update_layout(
    title='Single cycle revenue 2022',
    xaxis={'categoryorder':'total descending'},
    xaxis_title='',
    yaxis_title='€/MWh',
    yaxis_range=[56e3, 60e3],
    template='ggplot2',
    width=1200,
    height=500,
)
fig.update_coloraxes(showscale=False)
fig.update_traces(textfont_size=14, textposition='inside')

fig.write_image('../figures/single-cycle-rev.jpeg', scale=5, engine='orca')
fig

### Single cycle 2 hour

In [154]:
price_df

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI,am_pm
0,2022-01-01,2022-01-01 00:00:00,1,89.24,170.28,170.28,170.28,170.28,170.28,170.28,170.28,am
1,2022-01-01,2022-01-01 01:00:00,2,89.24,155.72,155.72,155.72,155.72,155.72,155.72,155.72,am
2,2022-01-01,2022-01-01 02:00:00,3,108.52,147.09,147.09,147.09,147.09,147.09,147.09,147.09,am
3,2022-01-01,2022-01-01 03:00:00,4,57.12,91.00,91.00,91.00,91.00,91.00,91.00,91.00,am
4,2022-01-01,2022-01-01 04:00:00,5,83.29,104.00,104.00,104.00,104.00,104.00,104.00,104.00,am
...,...,...,...,...,...,...,...,...,...,...,...,...
8754,2022-12-31,2022-12-31 19:00:00,20,191.16,333.00,333.00,333.00,333.00,333.00,333.00,333.00,pm
8755,2022-12-31,2022-12-31 20:00:00,21,192.85,310.00,310.00,310.00,310.00,310.00,310.00,310.00,pm
8756,2022-12-31,2022-12-31 21:00:00,22,194.88,270.00,270.00,270.00,270.00,270.00,270.00,270.00,pm
8757,2022-12-31,2022-12-31 22:00:00,23,214.28,217.78,217.78,217.78,217.78,217.78,217.78,217.78,pm


In [174]:
price_df.loc[price_df.groupby('date')['PUN'].apply(lambda x: x.isin(x.nlargest(2)))]

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI,am_pm
19,2022-01-01,2022-01-01 19:00:00,20,49.98,214.66,214.66,214.66,214.66,214.66,214.66,214.66,pm
20,2022-01-01,2022-01-01 20:00:00,21,-9.52,214.66,214.66,214.66,214.66,214.66,214.66,214.66,pm
41,2022-01-02,2022-01-02 17:00:00,18,251.07,245.00,245.00,245.00,245.00,245.00,245.00,245.00,pm
42,2022-01-02,2022-01-02 18:00:00,19,230.84,225.19,225.19,225.19,225.19,225.19,225.19,225.19,pm
65,2022-01-03,2022-01-03 17:00:00,18,308.24,300.00,300.00,300.00,300.00,300.00,300.00,300.00,pm
...,...,...,...,...,...,...,...,...,...,...,...,...
8728,2022-12-30,2022-12-30 17:00:00,18,138.81,222.95,222.95,222.95,222.95,222.95,222.95,222.95,pm
8730,2022-12-30,2022-12-30 19:00:00,20,195.24,229.64,229.64,229.64,229.64,229.64,229.64,229.64,pm
8752,2022-12-31,2022-12-31 17:00:00,18,191.05,333.00,333.00,333.00,333.00,333.00,333.00,333.00,pm
8753,2022-12-31,2022-12-31 18:00:00,19,186.31,333.00,333.00,333.00,333.00,333.00,333.00,333.00,pm


### Dual cycle

In [164]:
price_df['am_pm'] = price_df.hour.apply(lambda x: 'am' if x<= 12 else 'pm')
price_df_am_min = price_df.loc[price_df.am_pm == 'am'].groupby('date').min()
price_df_am_max = price_df.loc[price_df.am_pm == 'am'].groupby('date').max()
price_df_pm_min = price_df.loc[price_df.am_pm == 'pm'].groupby('date').min()
price_df_pm_max = price_df.loc[price_df.am_pm == 'pm'].groupby('date').max()
price_df_am_min.head()

,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI,am_pm
date,,,,,,,,,,,
2022-01-01,2022-01-01 00:00:00,1,-22.85,67.99,67.99,67.99,67.99,67.99,67.99,67.99,am
2022-01-02,2022-01-02 00:00:00,1,137.49,100.00,100.00,100.00,100.00,100.00,100.00,100.00,am
2022-01-03,2022-01-03 00:00:00,1,219.61,83.27,83.27,83.27,83.27,83.27,83.27,83.27,am
2022-01-04,2022-01-04 00:00:00,1,238.88,105.00,105.00,105.00,105.00,105.00,105.00,105.00,am
2022-01-05,2022-01-05 00:00:00,1,90.10,75.38,75.38,75.38,75.38,75.38,75.38,75.38,am


In [165]:
dual_cycle_dict = {
    'NORD': (price_df_am_max.NORD - price_df_am_min.PUN).sum() + (price_df_pm_max.NORD - price_df_pm_min.PUN).sum(),
    'CNOR': (price_df_am_max.CNOR - price_df_am_min.PUN).sum() + (price_df_pm_max.CNOR - price_df_pm_min.PUN).sum(),
    'CSUD': (price_df_am_max.CSUD - price_df_am_min.PUN).sum() + (price_df_pm_max.CSUD - price_df_pm_min.PUN).sum(),
    'SUD': (price_df_am_max.SUD - price_df_am_min.PUN).sum() + (price_df_pm_max.SUD - price_df_pm_min.PUN).sum(),
    'SICI': (price_df_am_max.SICI - price_df_am_min.PUN).sum() + (price_df_pm_max.SICI - price_df_pm_min.PUN).sum(),
    'SARD': (price_df_am_max.SARD - price_df_am_min.PUN).sum() + (price_df_pm_max.SARD - price_df_pm_min.PUN).sum(),
}

print(f'GB revenue: {(price_df_am_max.gb_price_eur - price_df_am_min.gb_price_eur).sum() + (price_df_pm_max.gb_price_eur - price_df_pm_min.gb_price_eur).sum()}')
dual_cycle_df = pd.DataFrame(dual_cycle_dict, index=['revenue']).transpose().round(2)
dual_cycle_df

GB revenue: 59433.55


,revenue
NORD,89550.08
CNOR,89582.56
CSUD,84005.95
SUD,82104.61
SICI,81879.03
SARD,82092.70


In [144]:
fig = px.bar(
    dual_cycle_df,
    color=dual_cycle_df.revenue.values,
    color_continuous_scale='solar',
    color_continuous_midpoint=87e3,
    text_auto='.3s'
)

fig.update_layout(
    title='Dual cycle revenue 2022',
    xaxis={'categoryorder':'total descending'},
    xaxis_title='',
    yaxis_title='€/MWh',
    yaxis_range=[80e3, 90e3],
    template='ggplot2',
    width=1200,
    height=500
)
fig.update_coloraxes(showscale=False)
fig.update_traces(textfont_size=14, textposition='inside')

fig.write_image('../figures/dual-cycle-rev.jpeg', scale=5, engine='orca')
fig